<a href="https://colab.research.google.com/github/Yunseokjin/Final-Team1/blob/main/%EA%B0%80%EC%84%A4_2_%22%EB%AC%B4%EB%A3%8C_%EC%9C%A0%EC%A0%80_vs_%EA%B5%AC%EB%A7%A4_%EC%9C%A0%EC%A0%80%2C_%EB%AC%B4%EC%97%87%EC%9D%84_%EB%8B%A4%EB%A5%B4%EA%B2%8C_%EB%B3%B4%EB%8A%94%EA%B0%80%3F_%ED%9A%8D%EB%93%9D_%EA%B2%BD%EB%A1%9C%EB%B3%84_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EC%84%9D%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 파일 읽기

import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/weighted_score_above_08_modified.csv')
df.head()

/tmp/ipython-input-264179111.py:1: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/weighted_score_above_08_modified.csv')


,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,is_outlier,genres1,genres2,genres3
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,2023-10-03 03:57:37,...,0.889438,0,1,0,0,1,False,Action,NaN,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,2023-09-30 17:55:55,...,0.914834,0,1,0,0,1,False,Action,NaN,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,2023-10-18 12:05:34,...,0.968375,3,1,0,0,1,True,Action,NaN,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,2023-10-25 19:10:11,...,0.826206,0,0,0,0,1,True,Action,NaN,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,2023-10-20 14:09:51,...,0.853612,0,1,0,0,1,True,Action,NaN,NaN


In [ ]:
api_isfree = pd.read_csv('/content/Appid_isfree.csv')
api_isfree.head()

,Unnamed: 0,appid,is_free
0,0,10,False
1,1,100,False
2,2,1000010,False
3,3,1000030,False
4,4,1000080,False


In [ ]:
print(api_isfree.shape)


(9606, 3)


In [ ]:
api_isfree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9606 entries, 0 to 9605
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9606 non-null   int64 
 1   appid       9606 non-null   int64 
 2   is_free     6777 non-null   object
dtypes: int64(2), object(1)
memory usage: 225.3+ KB


In [ ]:
# False 몇개인지 True 몇개인지 확인
api_isfree['is_free'].value_counts()

,count
is_free,
False,6653
True,124


# 가설 2: "무료 유저 vs 구매 유저, 무엇을 다르게 보는가? - 획득 경로별 리뷰 분석"

핵심 질문:
- received_for_free=True인 유저와
- steam_purchase=True인 유저는

게임을 평가하는 기준과 관점이 다를 것이다.

- **분석 방법**
    1. **데이터 분할:** received_for_free를 기준으로 두 그룹으로 나눈다.
    2. **패턴 비교 분석:**
        - **플레이타임 비교:** 두 그룹의 author_playtime_at_review 분포를 비교한다. (가설: 무료 유저는 이탈률이 더 높을 것이다.)
        - **긍정률 비교:** 단순히 voted_up 비율만 비교하는 것이 아니라, **어떤 장르의 게임에서 긍정률 차이가 크게 벌어지는지** 심층 분석한다.
        - **내용 비교 (NLP):** 두 그룹의 긍정/부정 리뷰에서 각각 핵심 키워드를 추출한다.
    - **기대 결과:** "구매 유저는 '가격 대비 가치', '환불' 등의 키워드를 사용하며 콘텐츠의 볼륨에 민감한 반면, 무료 유저는 '진입장벽', '과금 유도(P2W)' 등 게임의 첫 경험과 지속 가능성에 더 민감하다." 와 같은 인사이트 도출.

# 1.데이터 병합

데이터 병합: 먼저 weighted_score...csv와 Appid_isfree.csv를 appid 기준으로 left join 합니다. 이렇게 하면 is_free 정보가 없는 리뷰 데이터도 그대로 유지됩니다(해당 컬럼 값은 NaN이 됩니다).

In [ ]:
import pandas as pd

# 1. 두 데이터 파일 로드
review_df = pd.read_csv('weighted_score_above_08_modified.csv')
isfree_df = pd.read_csv('Appid_isfree.csv')

# 2. isfree_df 전처리 (결측치 처리 및 컬럼 정리)
isfree_df = isfree_df[['appid', 'is_free']].dropna() # 결측치 제거
isfree_df['is_free'] = isfree_df['is_free'].astype(bool) # boolean 형태로 변환

# 3. 'appid'를 기준으로 두 데이터프레임 병합
merged_df = pd.merge(review_df, isfree_df, on='appid', how='left')

# # 4. is_free 정보가 없는 게임은 유료(False)일 가능성이 높으므로 결측치를 False로 채워줌
# merged_df['is_free'] = merged_df['is_free'].fillna(False)

/tmp/ipython-input-538651769.py:4: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  review_df = pd.read_csv('weighted_score_above_08_modified.csv')


# 2.그룹나누기(무료,유료,불분명)

In [ ]:
# 'is_free'가 True인 그룹 (확실한 무료 게임)
group_A = merged_df[merged_df['is_free'] == True]

# 'is_free'가 False인 그룹 (확실한 유료 게임)
group_B = merged_df[merged_df['is_free'] == False]

# 'is_free'가 NaN인 그룹 (BM 불명확 게임)
group_C = merged_df[merged_df['is_free'].isna()]

# 각 그룹의 데이터 수 확인
print(f"그룹 A (무료 게임 리뷰 수): {len(group_A)}")
print(f"그룹 B (유료 게임 리뷰 수): {len(group_B)}")
print(f"그룹 C (BM 불명확 게임 리뷰 수): {len(group_C)}")

그룹 A (무료 게임 리뷰 수): 9638
그룹 B (유료 게임 리뷰 수): 165367
그룹 C (BM 불명확 게임 리뷰 수): 114269


# 3.NLP분석

## 분석 1차

In [ ]:
!pip install konlpy -q

In [ ]:
# Jieba 설치
!pip install jieba -q
# MeCab 설치
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8 -q
!pip install mecab-python3 -q

# ★★★★★ 바로 이 한 줄이 모든 문제를 해결합니다 ★★★★★
# MeCab의 설정 파일(mecabrc)을 파이썬 라이브러리가 찾는 위치에 바로가기(심볼릭 링크)로 연결합니다.
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Reading package lists...
Building dependency tree...
Reading state information...
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


In [ ]:
    !pip install stopwordsiso

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import jieba
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from stopwordsiso import stopwords
import warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 분석 대상 언어 목록 (리뷰 수 상위 8개)
TARGET_LANGUAGES = [
    'english', 'schinese', 'russian', 'spanish',
    'brazilian', 'turkish', 'koreana', 'japanese'
]

# stopwordsiso 라이브러리에서 사용할 언어 코드 매핑
LANG_CODE_MAP = {
    'english': 'en',
    'schinese': 'zh',
    'russian': 'ru',
    'spanish': 'es',
    'brazilian': 'pt', # brazilian portuguese는 pt로 처리
    'turkish': 'tr',
    'koreana': 'ko',
    'japanese': 'ja'
}

In [ ]:
# 형태소 분석기 인스턴스 생성
okt = Okt()
mecab = MeCab.Tagger("-Ochasen")

# ==============================================================================
# 2. 함수 정의: 언어별 텍스트 처리를 위한 함수들을 정의합니다.
# ==============================================================================

# 한국어 토크나이저: 명사, 동사, 형용사만 추출
def korean_tokenizer(text):
    tokens = okt.pos(text, stem=True) # 어간 추출
    return [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]

# 중국어 토크나이저
def chinese_tokenizer(text):
    return list(jieba.cut(text))

# 일본어 토크나이저: 명사, 동사, 형용사만 추출
def japanese_tokenizer(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        pos = node.feature.split(',')[0]
        if pos in ['名詞', '動詞', '形容詞']:
            tokens.append(node.surface)
        node = node.next
    return tokens

# 그 외 언어들을 위한 기본 토크나이저 (공백 기준 분리)
def default_tokenizer(text):
    return text.split()

In [ ]:
# ==============================================================================
# 3. 메인 분석 실행: 그룹별, 언어별로 키워드를 추출합니다.
# ==============================================================================

# 전체 결과를 저장할 딕셔너리
all_results = {}

# 3개의 그룹에 대해 분석 반복
groups_to_analyze = {
    'A (확실한 무료 게임)': group_A,
    'B (확실한 유료 게임)': group_B,
    'C (BM 불명확 게임)': group_C
}

for group_name, group_df in groups_to_analyze.items():
    print(f"==================== [ 그룹: {group_name} ] 분석 시작 ====================")
    group_results = {}

    # 8개 언어에 대해 분석 반복
    for lang in TARGET_LANGUAGES:
        print(f"\n--- [언어: {lang}] 키워드 추출 중 ---")

        # 1. 현재 언어의 '부정 리뷰' 데이터 필터링
        # ※※※※※ 중요: 'voted_up == 0'이 팀의 부정 리뷰 기준이 맞는지 확인해주세요! ※※※※※
        negative_reviews = group_df[(group_df['language'] == lang) & (group_df['voted_up'] == 0)]['review']

        # 리뷰가 없는 경우 건너뛰기
        if negative_reviews.empty:
            print("-> 해당 그룹/언어에 부정 리뷰가 없습니다.")
            continue
        # 최소 리뷰 수를 10으로 설정 (이 값은 조절 가능)
        MIN_REVIEW_COUNT = 10
        if len(negative_reviews) < MIN_REVIEW_COUNT:
            print(f"-> 리뷰 수가 {MIN_REVIEW_COUNT}개 미만이므로 분석을 건너뜁니다. (현재: {len(negative_reviews)}개)")
            continue # 다음 언어로 넘어감

        # 2. 언어에 맞는 토크나이저와 불용어 설정
        lang_code = LANG_CODE_MAP[lang]
        stop_words = list(stopwords(lang_code))

        if lang == 'koreana':
            tokenizer_func = korean_tokenizer
            # 한국어 불용어 추가 (필요시)
            stop_words.extend(['게임', '플레이', '유저', '하다', '있다', '되다', '없다', '이다', '같다', '그냥', '정말', '너무'])
        elif lang == 'schinese':
            tokenizer_func = chinese_tokenizer
        elif lang == 'japanese':
            tokenizer_func = japanese_tokenizer
        else: # english, russian 등
            tokenizer_func = default_tokenizer

        # 3. TF-IDF Vectorizer 생성 및 학습
        try:
            # 리뷰 수가 너무 적을 경우 min_df를 유연하게 조정
            min_doc_freq = 5 if len(negative_reviews) > 20 else 2 # 리뷰가 20개 이하면 min_df를 2로 낮춤

            tfidf_vectorizer = TfidfVectorizer(
                tokenizer=tokenizer_func,
                stop_words=stop_words,
                max_features=1000, # 가장 중요한 단어 1000개로 제한
                min_df=5,           # 최소 5개 문서 이상에 나타난 단어만 포함
                max_df=0.5          # 전체 문서의 50% 이상에 나타난 단어는 제외
            )
            tfidf_matrix = tfidf_vectorizer.fit_transform(negative_reviews)

            # 4. 상위 15개 키워드 추출
            feature_names = tfidf_vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1
            feature_scores = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
            top_keywords = [word for word, score in feature_scores[:15]]

            print(f"-> Top 15 키워드: {top_keywords}")
            group_results[lang] = top_keywords

        except Exception as e:
            print(f"-> 분석 중 오류 발생: {e}")

    all_results[group_name] = group_results
    print(f"==================== [ 그룹: {group_name} ] 분석 완료 ====================")


# 최종 결과를 보기 좋게 데이터프레임으로 변환 (선택 사항)
# pd.DataFrame(all_results).T```

==================== [ 그룹: A (확실한 무료 게임) ] 분석 시작 ====================

--- [언어: english] 키워드 추출 중 ---
-> Top 15 키워드: ['game.', 'play', '-', 'players', 'people', 'time', 'game,', 'fun', 'money', 'playing', 'it.', 'content', 'dlc', 'pay', 'cheaters']

--- [언어: schinese] 키워드 추출 중 ---
-> Top 15 키워드: ['玩家', '逼', '外挂', '玩', '傻', '封', '没有', '妈', '挂', '垃圾', '.', '买', '没', '中国', '蓝洞']

--- [언어: russian] 키워드 추출 중 ---
-> Top 15 키워드: ['-', 'игра', 'игру', 'играть', 'игре', 'игры', 'разработчики', 'стоит', 'либо', '2', '3', 'то,', 'игры.', '1', 'часов']

--- [언어: spanish] 키워드 추출 중 ---
-> Top 15 키워드: ['juego', 'dlc', 'jugar', 'juego,', 'base', 'comprar', 'ea', 'sistema', 'contenido', 'gente', 'hackers', 'falta', 'jugadores', 'recomiendo', 'puedes']

--- [언어: brazilian] 키워드 추출 중 ---
-> Top 15 키워드: ['pra', 'pubg', 'lite', '-', 'jogar', 'jogo,', 'dlcs', 'vc', 'comprar', 'reais', 'game', 'ea', 'melhor', 'jogos', 'jogo.']

--- [언어: turkish] 키워드 추출 중 ---
-> Top 15 키워드: ['oyun', 'oyunu', 'para', 'oyuna', '

## 분석 2차 (정규표현식언어 불용어처리)

In [ ]:
import re

In [ ]:
# 형태소 분석기 인스턴스 생성
okt = Okt()
mecab = MeCab.Tagger("-Ochasen")

# ==============================================================================
# 2. 함수 정의: 언어별 텍스트 처리를 위한 함수들을 정의합니다.
# ==============================================================================

# ★★★★★ 해결책 2: 텍스트 정제 함수 정의 ★★★★★
def clean_text(text):
    # 정규표현식을 사용하여 한국어, 영어, 중국어, 일본어, 숫자, 기본적인 공백을 제외한 모든 문자 제거
    # \u3040-\u30ff: 일본어 히라가나, 가타카나
    # \u4e00-\u9fff: 한자 (중국어, 일본어 공통)
    cleaned = re.sub(r'[^A-Za-z0-9가-힣\u3040-\u30ff\u4e00-\u9fff\s]', '', str(text))
    return cleaned.lower() # 소문자로 변환하여 일관성 유지

# 한국어 토크나이저: 명사, 동사, 형용사만 추출
def korean_tokenizer(text):
    tokens = okt.pos(text, stem=True) # 어간 추출
    return [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]

# 중국어 토크나이저
def chinese_tokenizer(text):
    return list(jieba.cut(text))

# 일본어 토크나이저: 명사, 동사, 형용사만 추출
def japanese_tokenizer(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        pos = node.feature.split(',')[0]
        if pos in ['名詞', '動詞', '形容詞']:
            tokens.append(node.surface)
        node = node.next
    return tokens

# 그 외 언어들을 위한 기본 토크나이저 (공백 기준 분리)
def default_tokenizer(text):
    return text.split()

In [ ]:
# ==============================================================================
# 3. 메인 분석 실행
# ==============================================================================

all_results = {}
groups_to_analyze = {
    'A (확실한 무료 게임)': group_A,
    'B (확실한 유료 게임)': group_B,
    'C (BM 불명확 게임)': group_C
}

for group_name, group_df in groups_to_analyze.items():
    print(f"==================== [ 그룹: {group_name} ] 분석 시작 ====================")
    group_results = {}

    for lang in TARGET_LANGUAGES:
        print(f"\n--- [언어: {lang}] 키워드 추출 중 ---")

        negative_reviews_series = group_df[(group_df['language'] == lang) & (group_df['voted_up'] == 0)]['review']

        MIN_REVIEW_COUNT = 10
        if len(negative_reviews_series) < MIN_REVIEW_COUNT:
            print(f"-> 리뷰 수가 {MIN_REVIEW_COUNT}개 미만이므로 분석을 건너뜁니다. (현재: {len(negative_reviews_series)}개)")
            continue

        cleaned_reviews = negative_reviews_series.apply(clean_text)

        lang_code = LANG_CODE_MAP[lang]
        stop_words = list(stopwords(lang_code))

        if lang == 'koreana':
            tokenizer_func = korean_tokenizer
            stop_words.extend(['게임', '플레이', '유저', '하다', '있다', '되다', '없다', '이다', '같다', '그냥', '정말', '너무'])
        elif lang == 'schinese':
            tokenizer_func = chinese_tokenizer
        elif lang == 'japanese':
            tokenizer_func = japanese_tokenizer
        else:
            tokenizer_func = default_tokenizer

        # ★★★★★ 여기부터가 완성된 try-except 블록입니다 ★★★★★
        try:
            min_doc_freq = 5 if len(cleaned_reviews) > 20 else 2

            tfidf_vectorizer = TfidfVectorizer(
                tokenizer=tokenizer_func,
                stop_words=stop_words,
                max_features=1000,
                min_df=min_doc_freq,
                max_df=0.5
            )
            tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_reviews)

            # 4. 상위 15개 키워드 추출
            feature_names = tfidf_vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1
            feature_scores = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
            top_keywords = [word for word, score in feature_scores[:15]]

            print(f"-> Top 15 키워드: {top_keywords}")
            group_results[lang] = top_keywords

        except Exception as e:
            # try 블록에서 오류가 발생했을 때 실행되는 부분
            print(f"-> 분석 중 오류 발생: {e}")

    # for lang 루프가 끝난 후, 다음 그룹으로 넘어가기 전에 결과를 저장합니다.
    all_results[group_name] = group_results
    print(f"==================== [ 그룹: {group_name} ] 분석 완료 ====================\n")

# 최종 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(all_results).T
print("\n\n===== 최종 분석 결과 요약 =====")
display(results_df) # 코랩에서는 display()가 더 예쁘게 보입니다.

==================== [ 그룹: A (확실한 무료 게임) ] 분석 시작 ====================

--- [언어: english] 키워드 추출 중 ---
-> Top 15 키워드: ['play', 'players', 'time', 'people', 'money', 'fun', 'games', 'cheaters', 'playing', 'dlc', 'content', '2', 'update', 'pay', 'devs']

--- [언어: schinese] 키워드 추출 중 ---
-> Top 15 키워드: ['玩家', '逼', '外挂', '玩', '傻', '没有', '妈', '挂', '封', '垃圾', '没', '买', '中国', '蓝洞', '服务器']

--- [언어: russian] 키워드 추출 중 ---
-> Top 15 키워드: ['2', '3', '1', '10', '5', 'pubg', '4', '500', 'h1', '100', 'b', '6', '20', 'warface', 'valve']

--- [언어: spanish] 키워드 추출 중 ---
-> Top 15 키워드: ['juego', 'dlc', 'jugar', 'comprar', 'jugadores', 'ea', 'base', 'ms', 'contenido', 'hackers', 'mierda', 'gente', 'sistema', 'recomiendo', 'falta']

--- [언어: brazilian] 키워드 추출 중 ---
-> Top 15 키워드: ['pubg', 'pra', 'lite', 'jogar', 'so', 'dlc', 'dlcs', 'game', 'j', 'melhor', 'reais', 'to', 'vc', 's', 'jogadores']

--- [언어: turkish] 키워드 추출 중 ---
-> Top 15 키워드: ['oyun', 'oyunu', 'ok', 'iin', 'para', 'oyunun', 'oyuna', 'pubg', 'a

,english,schinese,russian,spanish,brazilian,turkish,koreana,japanese
A (확실한 무료 게임),"[play, players, time, people, money, fun, game...","[玩家, 逼, 外挂, 玩, 傻, 没有, 妈, 挂, 封, 垃圾, 没, 买, 中国, 蓝...","[2, 3, 1, 10, 5, pubg, 4, 500, h1, 100, b, 6, ...","[juego, dlc, jugar, comprar, jugadores, ea, ba...","[pubg, pra, lite, jogar, so, dlc, dlcs, game, ...","[oyun, oyunu, ok, iin, para, oyunun, oyuna, pu...","[아니다, 핵, 겜, 서버, 중국인, 가다, 좋다, 내, 나오다, 싶다, 최적화, ...",NaN
B (확실한 유료 게임),"[play, games, time, fun, steam, 2, people, mon...","[逼, 傻, 没有, 玩, 玩家, 买, 没, 中文, 妈, 更新, 差评, 想, 中国, ...","[2, 3, 1, h1, 10, 4, 5, b, dlc, 20, 6, 15, 30,...","[ms, precio, est, jugar, pc, contenido, aos, d...","[voc, pra, preo, jogar, s, at, game, so, j, re...","[oyun, oyunu, ok, amk, oyunun, trke, tl, iin, ...","[아니다, 보다, 않다, 수, 만들다, 시발, 개, 좋다, 새끼, 버그, 내, 스팀...","[1, h, b, 日本語, 人, 的, 版, プレイ, 時間, 敵, てる, 開発, 点,..."
C (BM 불명확 게임),"[play, time, games, money, steam, people, fun,...","[玩家, 玩, 没有, 逼, 傻, 差评, 买, 没, 中国, 妈, 更新, 垃圾, 现在,...","[h1, 2, 3, 10, 1, 5, 4, b, list, i, code, stea...","[ms, jugar, precio, steam, dinero, est, gente,...","[voc, pra, jogar, s, at, game, so, j, to, est,...","[oyun, oyunu, iin, ok, oyunun, para, amk, oyun...","[도전, 과제, 아니다, 수, 보다, 않다, 개, 안, 거, 생각, 말, 레벨, 클...","[1, h, 版, 日本語, b, 人, 的, プレイ, 思い, 敵, 点, 方, てる, ..."


## 분석 3차 (의미없는 짧은 토큰 제거, HTML 태그 제거)

In [ ]:
# ==============================================================================
# 2. 함수 정의: "짧은 토큰 제거" 로직 추가
# ==============================================================================

# (clean_text 함수는 그대로 둡니다)
def clean_text(text):
    cleaned = re.sub(r'[^A-Za-z0-9가-힣\u3040-\u30ff\u4e00-\u9fff\s]', '', str(text))
    return cleaned.lower()

# 한국어 토크나이저 수정
def korean_tokenizer(text):
    tokens = okt.pos(text, stem=True)
    words = [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]
    return [word for word in words if len(word) > 1] # ★★★ 길이가 1 이하인 토큰은 제외

# 중국어 토크나이저 수정
def chinese_tokenizer(text):
    words = jieba.cut(text)
    return [word for word in words if len(word) > 1] # ★★★ 길이가 1 이하인 토큰은 제외

# 일본어 토크나이저 수정
def japanese_tokenizer(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        pos = node.feature.split(',')[0]
        if pos in ['名詞', '動詞', '形容詞']:
            # mecab은 원형(feature.split(',')[6])을 제공하므로 이를 사용하는 것이 더 좋습니다.
            original_form = node.feature.split(',')[6]
            # 원형이 '*'가 아닌 경우에만 추가 (기본형이 없는 기호 등 제외)
            if original_form != '*':
                tokens.append(original_form)
            else:
                tokens.append(node.surface)
        node = node.next
    return [word for word in tokens if len(word) > 1] # ★★★ 길이가 1 이하인 토큰은 제외

# 그 외 언어들을 위한 기본 토크나이저 수정
def default_tokenizer(text):
    words = text.split()
    return [word for word in words if len(word) > 1] # ★★★ 길이가 1 이하인 토큰은 제외

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import jieba
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from stopwordsiso import stopwords
import warnings
import warnings
warnings.filterwarnings('ignore')

# 1. 초기 설정
warnings.filterwarnings('ignore')

# 분석할 데이터프레임 (group_A, group_B, group_C가 이미 생성되었다고 가정)
# ...

# 분석 대상 언어 목록
TARGET_LANGUAGES = [
    'english', 'schinese', 'russian', 'spanish',
    'brazilian', 'turkish', 'koreana', 'japanese'
]

# 언어 코드 매핑
LANG_CODE_MAP = {
    'english': 'en', 'schinese': 'zh', 'russian': 'ru', 'spanish': 'es',
    'brazilian': 'pt', 'turkish': 'tr', 'koreana': 'ko', 'japanese': 'ja'
}

# 형태소 분석기 인스턴스 생성
okt = Okt()
mecab = MeCab.Tagger("-Ochasen")

# ==============================================================================
# 2. 함수 정의
# ==============================================================================

def clean_text(text):
    cleaned = re.sub(r'[^A-Za-z0-9가-힣\u3040-\u30ff\u4e00-\u9fff\s]', '', str(text))
    return cleaned.lower()

def korean_tokenizer(text):
    tokens = okt.pos(text, stem=True)
    return [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]

def chinese_tokenizer(text):
    return list(jieba.cut(text))

def japanese_tokenizer(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        pos = node.feature.split(',')[0]
        if pos in ['名詞', '動詞', '形容詞']:
            tokens.append(node.surface)
        node = node.next
    return tokens

def default_tokenizer(text):
    return text.split()

# ==============================================================================
# 3. 메인 분석 실행
# ==============================================================================

all_results = {}
groups_to_analyze = {
    'A (확실한 무료 게임)': group_A,
    'B (확실한 유료 게임)': group_B,
    'C (BM 불명확 게임)': group_C
}

for group_name, group_df in groups_to_analyze.items():
    print(f"==================== [ 그룹: {group_name} ] 분석 시작 ====================")
    group_results = {}

    for lang in TARGET_LANGUAGES:
        print(f"\n--- [언어: {lang}] 키워드 추출 중 ---")

        negative_reviews_series = group_df[(group_df['language'] == lang) & (group_df['voted_up'] == 0)]['review']

        MIN_REVIEW_COUNT = 10
        if len(negative_reviews_series) < MIN_REVIEW_COUNT:
            print(f"-> 리뷰 수가 {MIN_REVIEW_COUNT}개 미만이므로 분석을 건너뜁니다. (현재: {len(negative_reviews_series)}개)")
            continue

        cleaned_reviews = negative_reviews_series.apply(clean_text)

        lang_code = LANG_CODE_MAP[lang]
        stop_words = list(stopwords(lang_code))

        if lang == 'koreana':
            tokenizer_func = korean_tokenizer
            stop_words.extend(['게임', '플레이', '유저', '하다', '있다', '되다', '없다', '이다', '같다', '그냥', '정말', '너무'])
        elif lang == 'schinese':
            tokenizer_func = chinese_tokenizer
        elif lang == 'japanese':
            tokenizer_func = japanese_tokenizer
        else:
            tokenizer_func = default_tokenizer

        # ★★★★★ 여기부터가 완성된 try-except 블록입니다 ★★★★★
        try:
            min_doc_freq = 5 if len(cleaned_reviews) > 20 else 2

            tfidf_vectorizer = TfidfVectorizer(
                tokenizer=tokenizer_func,
                stop_words=stop_words,
                max_features=1000,
                min_df=min_doc_freq,
                max_df=0.5
            )
            tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_reviews)

            # 4. 상위 15개 키워드 추출
            feature_names = tfidf_vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1
            feature_scores = sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)
            top_keywords = [word for word, score in feature_scores[:15]]

            print(f"-> Top 15 키워드: {top_keywords}")
            group_results[lang] = top_keywords

        except Exception as e:
            # try 블록에서 오류가 발생했을 때 실행되는 부분
            print(f"-> 분석 중 오류 발생: {e}")

    # for lang 루프가 끝난 후, 다음 그룹으로 넘어가기 전에 결과를 저장합니다.
    all_results[group_name] = group_results
    print(f"==================== [ 그룹: {group_name} ] 분석 완료 ====================\n")

# 최종 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(all_results).T
print("\n\n===== 최종 분석 결과 요약 =====")
display(results_df) # 코랩에서는 display()가 더 예쁘게 보입니다.

==================== [ 그룹: A (확실한 무료 게임) ] 분석 시작 ====================

--- [언어: english] 키워드 추출 중 ---
-> Top 15 키워드: ['play', 'players', 'time', 'people', 'money', 'fun', 'games', 'cheaters', 'playing', 'dlc', 'content', '2', 'update', 'pay', 'devs']

--- [언어: schinese] 키워드 추출 중 ---
-> Top 15 키워드: ['玩家', '逼', '外挂', '玩', '傻', '没有', '妈', '挂', '封', '垃圾', '没', '买', '中国', '蓝洞', '服务器']

--- [언어: russian] 키워드 추출 중 ---
-> Top 15 키워드: ['2', '3', '1', '10', '5', 'pubg', '4', '500', 'h1', '100', 'b', '6', '20', 'warface', 'valve']

--- [언어: spanish] 키워드 추출 중 ---
-> Top 15 키워드: ['juego', 'dlc', 'jugar', 'comprar', 'jugadores', 'ea', 'base', 'ms', 'contenido', 'hackers', 'mierda', 'gente', 'sistema', 'recomiendo', 'falta']

--- [언어: brazilian] 키워드 추출 중 ---
-> Top 15 키워드: ['pubg', 'pra', 'lite', 'jogar', 'so', 'dlc', 'dlcs', 'game', 'j', 'melhor', 'reais', 'to', 'vc', 's', 'jogadores']

--- [언어: turkish] 키워드 추출 중 ---
-> Top 15 키워드: ['oyun', 'oyunu', 'ok', 'iin', 'para', 'oyunun', 'oyuna', 'pubg', 'a

,english,schinese,russian,spanish,brazilian,turkish,koreana,japanese
A (확실한 무료 게임),"[play, players, time, people, money, fun, game...","[玩家, 逼, 外挂, 玩, 傻, 没有, 妈, 挂, 封, 垃圾, 没, 买, 中国, 蓝...","[2, 3, 1, 10, 5, pubg, 4, 500, h1, 100, b, 6, ...","[juego, dlc, jugar, comprar, jugadores, ea, ba...","[pubg, pra, lite, jogar, so, dlc, dlcs, game, ...","[oyun, oyunu, ok, iin, para, oyunun, oyuna, pu...","[아니다, 핵, 겜, 서버, 중국인, 가다, 좋다, 내, 나오다, 싶다, 최적화, ...",NaN
B (확실한 유료 게임),"[play, games, time, fun, steam, 2, people, mon...","[逼, 傻, 没有, 玩, 玩家, 买, 没, 中文, 妈, 更新, 差评, 想, 中国, ...","[2, 3, 1, h1, 10, 4, 5, b, dlc, 20, 6, 15, 30,...","[ms, precio, est, jugar, pc, contenido, aos, d...","[voc, pra, preo, jogar, s, at, game, so, j, re...","[oyun, oyunu, ok, amk, oyunun, trke, tl, iin, ...","[아니다, 보다, 않다, 수, 만들다, 시발, 개, 좋다, 새끼, 버그, 내, 스팀...","[1, h, b, 日本語, 人, 的, 版, プレイ, 時間, 敵, てる, 開発, 点,..."
C (BM 불명확 게임),"[play, time, games, money, steam, people, fun,...","[玩家, 玩, 没有, 逼, 傻, 差评, 买, 没, 中国, 妈, 更新, 垃圾, 现在,...","[h1, 2, 3, 10, 1, 5, 4, b, list, i, code, stea...","[ms, jugar, precio, steam, dinero, est, gente,...","[voc, pra, jogar, s, at, game, so, j, to, est,...","[oyun, oyunu, iin, ok, oyunun, para, amk, oyun...","[도전, 과제, 아니다, 수, 보다, 않다, 개, 안, 거, 생각, 말, 레벨, 클...","[1, h, 版, 日本語, b, 人, 的, プレイ, 思い, 敵, 点, 方, てる, ..."


## 분석 4차 (숫자 제거, 깨진 단어 제거)

In [ ]:
import pandas as pd
from konlpy.tag import Okt
import jieba
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from stopwordsiso import stopwords
import warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. 초기 설정
warnings.filterwarnings('ignore')

# 분석할 데이터프레임 (group_A, group_B, group_C가 이미 생성되었다고 가정)
# ...

# 분석 대상 언어 목록
TARGET_LANGUAGES = [
    'english', 'schinese', 'russian', 'spanish',
    'brazilian', 'turkish', 'koreana', 'japanese'
]

# 언어 코드 매핑
LANG_CODE_MAP = {
    'english': 'en', 'schinese': 'zh', 'russian': 'ru', 'spanish': 'es',
    'brazilian': 'pt', 'turkish': 'tr', 'koreana': 'ko', 'japanese': 'ja'
}

# 형태소 분석기 인스턴스 생성
okt = Okt()
mecab = MeCab.Tagger("-Ochasen")

In [ ]:
# ==============================================================================
# 2. 함수 정의
# ==============================================================================

def clean_text(text):
    cleaned = re.sub(r'[^A-Za-z0-9가-힣\u3040-\u30ff\u4e00-\u9fff\s]', '', str(text))
    return cleaned.lower()

def korean_tokenizer(text):
    tokens = okt.pos(text, stem=True)
    return [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective']]

def chinese_tokenizer(text):
    return list(jieba.cut(text))

def japanese_tokenizer(text):
    node = mecab.parseToNode(text)
    tokens = []
    while node:
        pos = node.feature.split(',')[0]
        if pos in ['名詞', '動詞', '形容詞']:
            tokens.append(node.surface)
        node = node.next
    return tokens

def default_tokenizer(text):
    return text.split()


In [ ]:
# ==============================================================================
# 3. 메인 분석 실행 (최종 정제 로직 포함)
# ==============================================================================

all_results = {}
groups_to_analyze = {
    'A (확실한 무료 게임)': group_A,
    'B (확실한 유료 게임)': group_B,
    'C (BM 불명확 게임)': group_C
}

for group_name, group_df in groups_to_analyze.items():
    print(f"==================== [ 그룹: {group_name} ] 분석 시작 ====================")
    group_results = {}

    for lang in TARGET_LANGUAGES:
        print(f"\n--- [언어: {lang}] 키워드 추출 중 ---")

        negative_reviews_series = group_df[(group_df['language'] == lang) & (group_df['voted_up'] == 0)]['review']

        MIN_REVIEW_COUNT = 10
        if len(negative_reviews_series) < MIN_REVIEW_COUNT:
            print(f"-> 리뷰 수가 {MIN_REVIEW_COUNT}개 미만이므로 분석을 건너뜁니다. (현재: {len(negative_reviews_series)}개)")
            continue

        # ★★★ 1단계: 정규식 강화 (숫자 제거) ★★★
        def clean_text_final(text):
            # 오직 '문자'와 '공백'만 남깁니다. (숫자 제거)
            cleaned = re.sub(r'[^A-Za-z가-힣\u3040-\u30ff\u4e00-\u9fff\s]', '', str(text))
            return cleaned.lower()

        cleaned_reviews = negative_reviews_series.apply(clean_text_final)

        # 2단계: 토크나이저 선택 및 기본 불용어 설정
        lang_code = LANG_CODE_MAP[lang]
        stop_words = list(stopwords(lang_code))

        if lang == 'koreana':
            tokenizer_func = korean_tokenizer
        elif lang == 'schinese':
            tokenizer_func = chinese_tokenizer
        elif lang == 'japanese':
            tokenizer_func = japanese_tokenizer
        else:
            tokenizer_func = default_tokenizer

        # ★★★ 3단계: 사용자 정의 불용어 추가 ★★★
        # 분석 결과를 보면서 의미 없는 단어들을 여기에 추가합니다.
        custom_stopwords = []
        if lang == 'russian':
            custom_stopwords.extend(['h1', 'ps'])
        elif lang == 'turkish':
            custom_stopwords.extend(['ok', 'iin', 'artk', 'bi', 'u']) # 깨진 단어 및 무의미한 단어
        elif lang == 'brazilian':
            custom_stopwords.extend(['pra', 'vc', 'so', 'to', 'est', 'pro'])

        stop_words.extend(custom_stopwords)

        # TF-IDF 분석
        try:
            tfidf_vectorizer = TfidfVectorizer(
                # tokenizer는 그대로 사용 (짧은 단어 1차 제거)
                tokenizer=tokenizer_func,
                stop_words=stop_words,
                max_features=1000,
                min_df=5, # min_df는 다시 5 정도로 안정적으로 설정
                max_df=0.5
            )
            tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_reviews)

            feature_names = tfidf_vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1

            # ★★★ 추가 필터링: 숫자만으로 구성된 토큰 제거 ★★★
            final_scores = []
            for name, score in zip(feature_names, scores):
                if not name.isdigit(): # 토큰이 숫자로만 이루어져 있지 않은 경우에만
                    final_scores.append((name, score))

            feature_scores = sorted(final_scores, key=lambda x: x[1], reverse=True)
            top_keywords = [word for word, score in feature_scores[:15]]

            print(f"-> Top 15 키워드: {top_keywords}")
            group_results[lang] = top_keywords

        except Exception as e:
            print(f"-> 분석 중 오류 발생: {e}")

    all_results[group_name] = group_results
    print(f"==================== [ 그룹: {group_name} ] 분석 완료 ====================\n")

# 최종 결과 확인
results_df = pd.DataFrame(all_results).T
print("\n\n===== 최종 분석 결과 요약 =====")
display(results_df)

==================== [ 그룹: A (확실한 무료 게임) ] 분석 시작 ====================

--- [언어: english] 키워드 추출 중 ---
-> Top 15 키워드: ['play', 'players', 'time', 'money', 'people', 'fun', 'games', 'cheaters', 'dlc', 'playing', 'content', 'update', 'pubg', 'pay', 'devs']

--- [언어: schinese] 키워드 추출 중 ---
-> Top 15 키워드: ['玩家', '逼', '外挂', '玩', '傻', '没有', '妈', '挂', '封', '垃圾', '没', '买', '中国', '蓝洞', '服务器']

--- [언어: russian] 키워드 추출 중 ---
-> Top 15 키워드: ['pubg', 'warface', 'valve', 'b', 'h', 'dlc', 'upd', 'i', 'u', 'fps', 'to', 'steam', 'pay', 'destiny', 'win']

--- [언어: spanish] 키워드 추출 중 ---
-> Top 15 키워드: ['juego', 'dlc', 'jugar', 'comprar', 'jugadores', 'ea', 'base', 'ms', 'contenido', 'hackers', 'mierda', 'gente', 'sistema', 'recomiendo', 'falta']

--- [언어: brazilian] 키워드 추출 중 ---
-> Top 15 키워드: ['pubg', 'lite', 'jogar', 'dlc', 'dlcs', 'j', 'game', 'melhor', 'reais', 's', 'h', 'jogadores', 'ea', 'contedo', 'comprar']

--- [언어: turkish] 키워드 추출 중 ---
-> Top 15 키워드: ['oyun', 'oyunu', 'oyunun', 'para', 'oyuna'

,english,schinese,russian,spanish,brazilian,turkish,koreana,japanese
A (확실한 무료 게임),"[play, players, time, money, people, fun, game...","[玩家, 逼, 外挂, 玩, 傻, 没有, 妈, 挂, 封, 垃圾, 没, 买, 中国, 蓝...","[pubg, warface, valve, b, h, dlc, upd, i, u, f...","[juego, dlc, jugar, comprar, jugadores, ea, ba...","[pubg, lite, jogar, dlc, dlcs, j, game, melhor...","[oyun, oyunu, oyunun, para, oyuna, pubg, amk, ...","[하다, 게임, 핵, 겜, 아니다, 중국인, 없다, 유저, 서버, 좋다, 가다, 내...",NaN
B (확실한 유료 게임),"[play, games, time, fun, steam, people, money,...","[逼, 傻, 没有, 玩, 玩家, 买, 没, 中文, 妈, 更新, 差评, 想, 中国, ...","[h, b, dlc, upd, steam, i, the, of, list, ea, ...","[ms, precio, est, jugar, pc, contenido, aos, d...","[voc, preo, jogar, s, at, game, reais, j, pc, ...","[oyun, oyunu, amk, tl, oyunun, trke, oyuna, gz...","[되다, 없다, 아니다, 보다, 않다, 이다, 수, 만들다, 시발, 새끼, 버그, ...","[h, b, 的, 人, 日本語, プレイ, 版, 時間, 敵, てる, 開発, 点, 出来..."
C (BM 불명확 게임),"[play, time, games, money, steam, people, fun,...","[玩家, 玩, 没有, 逼, 傻, 差评, 买, 没, 中国, 妈, 更新, 垃圾, 现在,...","[h, b, steam, list, i, code, u, the, of, dlc, ...","[ms, jugar, precio, steam, dinero, est, juegos...","[voc, jogar, s, at, game, j, dinheiro, preo, j...","[oyun, oyunu, oyunun, para, amk, oyuna, oyunda...","[도전, 과제, 아니다, 수, 보다, 않다, 개, 이다, 안, 플레이, 거, 생각,...","[h, 版, 日本語, 人, b, 的, プレイ, t, 思い, 敵, 点, 方, てる, ..."


# 인사이트

## 결과 분석

[결과에 대한 종합 평가: "성공적인 분석"]

1. 그룹별 특징이 뚜렷하게 나타남:
    - 그룹 A (무료 게임): cheaters(부정행위자), 外挂(핵), 핵, 서버 등 '공정한 플레이 환경' 에 대한 불만이 두드러집니다. 또한, dlc나 pay 같은 키워드를 통해 무료 게임의 '수익 모델(BM)' 에 대한 불만도 명확히 보입니다.
    - 그룹 B (유료 게임): price(가격), precio(가격), preo(가격), tl(터키 리라) 등 '가격' 관련 키워드가 여러 언어에서 공통적으로 나타납니다. story, 剧情(줄거리), historia(역사/이야기) 등 '콘텐츠의 깊이' 에 대한 평가도 중요한 요소임을 알 수 있습니다.
    - 그룹 C (BM 불명확): price, dinero, preo 등 가격 관련 키워드가 뚜렷하게 나타납니다. 이는 그룹 C가 '유료 게임(그룹 B)'의 특성을 강하게 띤다는 우리의 가설을 강력하게 뒷받침하는 결정적인 증거입니다.

2. 문화권별 특성 유지:
    - 가설 3에서 발견했던 문화권별 특징이 여전히 잘 유지되고 있습니다.
        - 서구/라틴권: money, price, dlc (가격 대비 가치)
        - 한국/중국: 핵, 外挂 (외부 요인, 특히 불법 프로그램)
        - 일본: プレイ(플레이), 時間(시간) (플레이 경험 자체)

3. 정제 효과 확인:

    - 대부분의 언어에서 한 글자짜리 무의미한 토큰이 사라졌습니다. 러시아어 등에서 h, b 등이 일부 남았지만, 이는 아마도 h1z1 같은 특정 게임 이름이나 문법적 요소가 완전히 걸러지지 않은 것으로 보입니다. 하지만 이 정도는 전체적인 인사이트를 해석하는 데 큰 지장을 주지 않는 수준입니다. 완벽한 정제보다 중요한 것은 "유의미한 차이를 발견하는 것"이며, 이미 그 목표는 달성했습니다.


## 스토리텔링

1. 서론: 문제 제기

    "저희는 '스팀 리뷰 데이터에서 유저의 진짜 목소리를 어떻게 들을 수 있을까?'라는 질문에서 출발했습니다. 특히, 게임의 비즈니스 모델(BM)에 따라 유저 경험이 어떻게 달라지는지 확인하기 위해, 리뷰를 '확실한 무료 게임(A)', '확실한 유료 게임(B)', 그리고 'BM이 불명확한 게임(C)' 세 그룹으로 나누어 부정 리뷰의 핵심 키워드를 심층 분석했습니다."

2. 본론 1: 무료 게임 유저의 목소리 - "공정한 운동장을 원한다"

    (그룹 A 결과 제시)
    "무료 게임 유저들의 가장 큰 불만은 '돈' 그 자체가 아니었습니다. 한국어의 '핵', '중국인', 영어의 'cheaters', 중국어의 '外挂(핵)' 키워드가 보여주듯, 이들은 '불공정한 플레이 환경' 에 가장 크게 분노했습니다. 또한 dlc, pay와 같은 키워드를 통해, 무료로 시작했지만 결국 돈을 써야만 하는 'Pay-to-Win' 시스템에 대한 거부감도 뚜렷하게 나타났습니다."

3. 본론 2: 유료 게임 유저의 목소리 - "내 돈의 가치를 증명하라"

    (그룹 B 결과 제시)
    "반면, 유료 게임 유저들의 불만은 명확했습니다. 스페인어, 브라질어, 터키어 등 여러 언어에서 공통적으로 '가격(precio, preço, tl)' 관련 키워드가 최상위에 올랐습니다. 이들은 자신이 지불한 돈에 합당한 '가치(Value for Money)' 를 원했습니다. story, 剧情 키워드는 이들이 단순한 플레이를 넘어, '잘 짜인 콘텐츠' 를 중요한 평가 기준으로 삼고 있음을 보여줍니다."

4. 본론 3: 가설 검증 - 안갯속의 유저들은 누구인가?
    
    (그룹 C 결과 제시)
    "분석의 신뢰도를 높이기 위해, 저희는 BM이 불명확했던 그룹 C를 독립적으로 분석했습니다. 놀랍게도, 그룹 C의 키워드 패턴은 무료 게임 그룹(A)이 아닌, 유료 게임 그룹(B)과 거의 일치했습니다. price, dinero 와 같은 가격 키워드의 등장은, 이 그룹의 대다수가 유료 게임 유저임을 강력하게 시사합니다. 이를 통해 저희의 핵심 분석(A vs B)이 매우 타당함을 교차 검증할 수 있었습니다."

5. 결론: 타겟에 맞는 전략적 소통의 필요성
    
    "결론적으로, 모든 유저는 '게임이 재미없다'고 말하지만, 그 이유는 전혀 다릅니다. F2P 게임사는 '핵 방지'와 '공정한 과금 모델' 에 대한 소통을 최우선으로 해야 하며, 패키지 게임사는 '가격 정책'과 '콘텐츠 볼륨' 에 대한 유저들의 기대를 충족시켜야 합니다. 이처럼 유저 그룹의 목소리를 명확히 구분하여 듣는 것이 성공적인 게임 운영의 첫걸음이 될 것입니다."

## 1. 얼마나 '강하게' 구분이 되는가? - "전혀 다른 세계의 이야기"

[핵심 불만 주제(Thematic Keyword) 비교]


|불만 주제	|그룹 A: 무료 게임 (Free-to-Play)	|그룹 B: 유료 게임 (Pay-to-Play)	|해석: 분노의 근원|
|--|--|--|--|
|환경 문제	|핵(ko), cheaters(en), 外挂(zh), hackers(es)|	(거의 없음)	|"내 시간을 망치는 외부 방해 요소를 막아달라!"|
|비용 문제	|dlc, pay, money|	price(en), precio(es), preo(pt), tl(tr)	|"계속 돈을 내게 만들지 마라!" vs "처음에 낸 돈이 아깝다!"|
|콘텐츠 문제	|update, dlc| (새로운 것의 부재)	story(en), 剧情(zh), historia(es)|	"즐길 거리가 부족하다!" vs "기본적인 이야기가 부실하다!"|
|개발사 문제	|蓝洞(Bluehole) (특정 게임사)|	devs (개발자 일반)|	"이 회사 운영이 문제다!" vs "개발자들이 일을 안 한다!"|

## 2. 무료 게임의 정체: "부분 유료화 슈팅 게임"이라는 관찰
이 관찰은 정말 정확합니다. A그룹의 키워드들은 특정 장르, 특히 글로벌 F2P(부분 유료화) 슈팅 게임의 특징을 너무나도 명확하게 보여주고 있습니다.

[근거가 되는 키워드들]
- 직접적인 게임/개발사 이름: pubg, lite, warface, destiny, 蓝洞(Bluehole, 펍지 개발사)
- 장르 관련 용어: cheaters, 핵, 外挂, hackers, fps, optimizasyon(최적화), 서버
- F2P BM 관련 용어: pay, dlc (무료 게임에서의 추가 결제)

"스팀에서 '무료 게임'에 대한 부정적 여론을 주도하는 것은 F2P 슈팅 장르이며, 이들의 핵심 불만은 '핵'과 '서버 문제' 이다."

가장 목소리가 큰 F2P 슈팅 유저들의 불만부터 해결해야 한다